`conda activate imgpro`

In [10]:
import os
import tifffile
import sys
import importlib
import numpy as np
import cv2
import imageio
import json

In [36]:
def make_rgb(frame, rmax, gmax, shape=(512, 512, 3)):
    """Creates an RGB image from a frame."""
    gfp, rfp = frame
    rgb = np.zeros(shape, np.ubyte)
    adjust = lambda frame, lo, hi: np.clip((frame.astype(np.float32) - lo) / (hi - lo), 0, 1)
    rgb[..., 0] = adjust(rfp, 0, rmax) * 255
    rgb[..., 1] = adjust(gfp, 0, gmax) * 255
    return rgb

def overlay_text(rgb, frame_num, time_sec):
    """Overlays 'Frame: X, Time: Y sec' text on the top-left of the image."""
    text = f'Volume: {frame_num}, Time: {time_sec:.2f} sec'
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.75
    color = (255, 255, 255)
    thickness = 2
    position = (10, 25)
    cv2.putText(rgb, text, position, font, font_scale, color, thickness, cv2.LINE_AA)
    return rgb

def create_video(out_dir, tif_dir, mp4_path, frame_index, time_vector, fps=6, quality=10, rmax=1200, gmax=170):
    """Creates an MP4 file from a MIP TIF file with frame/time overlay."""
    mip = tifffile.imread(os.path.join(out_dir, 'mip_' + tif_dir + '.tif')) # loads as (time/frame, 2, height, width)
    with imageio.get_writer(mp4_path, fps=fps, codec='mjpeg', quality=quality, pixelformat='yuvj444p') as mp4:
        for i, frame in enumerate(mip):
            rgb = make_rgb(frame, rmax, gmax)
            rgb = overlay_text(rgb, frame_index[i], time_vector[i])
            mp4.append_data(rgb)

In [37]:
input_dir = r'D:\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001'
tif_dir = 'bleach_corrected'
mp4_path = os.path.join(input_dir, 'mip_with_frame_time' + '.mp4')

# load metadata
with open(os.path.join(input_dir, 'metadata.json'), 'r') as f:
    metadata = json.load(f)
frame_index = metadata['frame_index'] # list of ints
fps = metadata['fps'] # int
time_vector = np.array(frame_index) / fps # in seconds

create_video(input_dir, tif_dir, mp4_path, frame_index, time_vector)